## **PIPELINE**

Se realiza el pipelin con el objetivo de *empaquetar* todas las acciones que van desde el preprocesamiento de los datos hasta las predicciones del modelo en un único objeto.
Para esto se usa la libreria de Scikit-learn llamada pipeline. 
El mismo incluye los siguientes pasos del flujo del preprocesamiento de datos:
- limpieza - ver EDA
- normalización - ver EDA
- transformación
- reducción de dimensionalidad
- ingeniería de features - ver EDA
- modelos predictivos 
- elección del modelo predictivo


- se importan las librerias

In [14]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import joblib 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

- creo 3 pipelines, lo que me interesa ver es que es más performante, si el arbol, el k vecinos o ambos

In [ ]:
pipeline1= Pipeline([('knn', KNeighborsClassifier(random_state=42))])

In [ ]:
pipeline2= Pipeline([('clf', DecisionTreeClassifier(random_state=42))])

In [ ]:
both_pipeline= Pipeline([('knn', KNeighborsClassifier(random_state=42)), 
			('clf', DecisionTreeClassifier(random_state=42))])

In [ ]:
my_pipeline=[pipeline1, pipeline2, both_pipeline]

# Ingesta de datos

In [ ]:
df_train=pd.read_csv('Datathon\\hospitalizaciones_train.csv', encoding='utf-8')

In [ ]:
df=df_train.copy()

# Limpieza / Normalización / Transformación

In [ ]:
#se crea una funcion para transformar la columna y a una variable dicotómica 
# en 1 y 0 que representa si la estadía será larga (0) o corta (1)

def transf_label(df):
    lista_dias=df['Stay (in days)']
    lista_estancia=[]
    for i in lista_dias:
        if i <=8: 
            i=0
        elif i>8:
            i=1
        lista_estancia.append(i)
    array_estancia=np.array(lista_estancia)
    array_estancia.reshape(1,-1)
    array_estancia.shape
    df['estancia']=array_estancia
    df.drop(columns=['Stay (in days)'], inplace=True)
    return df

In [ ]:
#aplico la funcion
transf_label(df)

In [ ]:
#transformo las columnas categoricas a valores numéricos para poder aplicar las modelos
le = preprocessing.LabelEncoder()
df['Insurance'] = le.fit_transform(df['Insurance'])
df['health_conditions'] = le.fit_transform(df['health_conditions'])
df['Severity of Illness'] = le.fit_transform(df['Severity of Illness'])
df['Type of Admission'] = le.fit_transform(df['Type of Admission'])
df['gender'] = le.fit_transform(df['gender'])
df['Age'] = le.fit_transform(df['Age'])
df['doctor_name'] = le.fit_transform(df['doctor_name'])
df['Ward_Facility_Code'] = le.fit_transform(df['Ward_Facility_Code'])
df['Department'] = le.fit_transform(df['Department'])

# Reducción de dimensionalidad

In [ ]:
y=df[['estancia']]
#x=df[['Available Extra Rooms in Hospital','Department','Ward_Facility_Code','doctor_name','staff_available','patientid','Age','gender','Type of Admission','Severity of Illness','health_conditions','Visitors with Patient','Insurance','Admission_Deposit']]
X=df[['Department','Age']]

In [15]:
# Separaro los datos en train y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelos predictivos 

In [21]:
# Hacemos un diccionario para fines organizativos

pipe_dict = {0: 'k-vecinos', 1: 'Árbol de decisión'}

In [22]:
# Entrenamos

for pipe in my_pipeline:
	pipe.fit(X_train, y_train)

In [23]:
# Evaluamos

for idx, val in enumerate(my_pipeline):
	print('%s pipeline accuracy en test: %.3f' % (pipe_dict[idx], val.score(X_test, y_test)))

Regresión Logística pipeline accuracy en test: 0.900
SVM pipeline accuracy en test: 0.900
Árbol de decisión pipeline accuracy en test: 0.867


# Elección del modelo predictivo

In [24]:
# Identificamos el mejor modelo para el set de testeo

best_acc = 0.0
best_clf = 0
best_pipe = ''
for idx, val in enumerate(my_pipeline):
	if val.score(X_test, y_test) > best_acc:
		best_acc = val.score(X_test, y_test)
		best_pipe = val
		best_clf = idx
print('Modelo con el mejor accuracy: %s' % pipe_dict[best_clf])

Modelo con el mejor accuracy: Regresión Logística


In [25]:
# Guardo el pipeline en un archivo

joblib.dump(best_pipe, 'Mejor_pipeline.pkl', compress=1)
print('Pipeline de %s guardado a archivo' % pipe_dict[best_clf])

Pipeline de Regresión Logística guardado a archivo


In [27]:
best_model = joblib.load('Mejor_pipeline.pkl')


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42) 

# datos, dejo un 20% para testear




- - -

In [29]:
best_model.fit(X_train,y_train)

Pipeline(steps=[('scl', StandardScaler()), ('pca', PCA(n_components=2)),
                ('clf', LogisticRegression(random_state=42))])

In [30]:
best_model.score(X_test, y_test)

0.9333333333333333

In [ ]:
#EXACTITUD
exactitud = accuracy_score(y_test, y_test_pred)

#PRECISION
precision = precision_score(y_test, y_test_pred, average= None)

#EXAHUSTIVIDAD/RECALL/SENSIBILIDAD
recall1= recall_score(y_test, y_test_pred, average= None)

#F1-SCORE
precision1 = f1_score(y_test, y_test_pred, average= None)

print(f'El valor de la precisión para este modelo es de {precision}')
print(f'El valor de la recall es de {recall1}')
print(f'El valor de accuracy es de {exactitud}')
print(f'El f1-score es de:{precision1}, type{precision1}')